# Realign two EEGLab.Epoch object's epochs.

@Author [@FranckPrts](hstate_dictps://github.com/FranckPrts).

Here we provide a python-based solution to **realign concurrent epochs contained in two epoched object that lost their alignement because of losing their concurrence during preprocessing.**

After procedding to preprocessing in EEGLAB, the data were saved in `.set` format. When later loading this data, the 

Our main goal here is to remove epochs in a given EEG that should have been removed during preprocessing when the concurent epoch in the other EEG was. At the end, we should have two epoched EEG with the same amount of epochs and where each epoch at a given index correspond what was curcurent while recording.

As exemplified below: 

#IMAGE

Things that might be idiosynchratic:

1. Our EEG data was segmented in 1sec epochs (which helps with congruency between epochs and the time they where collected).
2. Preprocessing was done independelty for each EEG data. 
3. Each dyad were preprocessed 2-3 times (iteration).
    - epochs ID that were rejected were noted in a separate file between each round.
    - the objects were save at the end of each iteration and re-read after so .

Our issue arise from the fact that once each step was performed, saving the data would lead to losing track of what was the epochs original IDs. 

In that process we see that an epoch that originally had the ID #6 can end up with the new ID #3. 

To retrieve the original id of the epoch, we will have to work bakward from the last iteration of preprocessing to the first iteration. At each step we will store what was the previous ID of the epochs so we can find their original IDs. 

## Imports

In [1]:
# Package 
import mne
import numpy as np
import pandas as pd

# Custom functions
from utils import align_utils

# %matplotlib inline

We import two eeg stream that were preprocessed in MATLAB

In [2]:
files_to_process = np.loadtxt("files_to_process.csv",
                 delimiter=",", dtype=str)

dyad = [x for x in files_to_process]
# Careful, the file_to_process is in the order (dyad_nb, eeg_filepath_child, eeg_filepath_adutl)
dy = dyad[0]
data_path = '../FINS-data/'

In [3]:
eeg_child = align_utils.EpochsEEGLAB_to_mneEpochsFIF('{}{}_{}_FP/{}'.format(data_path, dy[0], 'child', dy[1])) 
eeg_adult = align_utils.EpochsEEGLAB_to_mneEpochsFIF('{}{}_{}_FP/{}'.format(data_path, dy[0], 'adult', dy[2]))

Extracting parameters from /Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/../FINS-data/213_child_FP/FINS_213_Child_FreePlay_Filt_xchan_epochs_rej_prunedICA_rej2.set...
Not setting metadata
325 matching events found
No baseline correction applied
0 projection items activated
Ready.
Reading /var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/tmpvv3n5bu9tmp.fif ...
    Found the data of interest:
        t =       0.00 ...     998.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
325 matching events found
No baseline correction applied
0 projection items activated


/Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/utils/align_utils.py:29: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  tmp = mne.io.read_epochs_eeglab(path)
/Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/utils/align_utils.py:33: RuntimeWarning: This filename (/var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/tmpvv3n5bu9tmp.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  tmp.save(tmpdir+"tmp.fif", overwrite=True, verbose=None)
/Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/utils/align_utils.py:36: RuntimeWarning: This filename (/var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/tmpvv3n5bu9tmp.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  return mne.read_epochs(tmpdir+"tmp.fif")


Extracting parameters from /Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/../FINS-data/213_adult_FP/FINS_213_ADULT_FreePlay_Rej_ICA_rej2.set...
Not setting metadata
316 matching events found
No baseline correction applied
0 projection items activated
Ready.
Reading /var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/tmphek5udm4tmp.fif ...
    Found the data of interest:
        t =       0.00 ...     998.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
316 matching events found
No baseline correction applied
0 projection items activated


/Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/utils/align_utils.py:29: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  tmp = mne.io.read_epochs_eeglab(path)
/Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/utils/align_utils.py:33: RuntimeWarning: This filename (/var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/tmphek5udm4tmp.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  tmp.save(tmpdir+"tmp.fif", overwrite=True, verbose=None)
/Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/utils/align_utils.py:36: RuntimeWarning: This filename (/var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/tmphek5udm4tmp.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  return mne.read_epochs(tmpdir+"tmp.fif")


Let's see how many epochs we have per EEG file:

In [4]:
print('EEG-child has {} epochs.'.format(eeg_child.get_data().shape[0]))
print('EEG-adult has {} epochs.'.format(eeg_adult.get_data().shape[0]))

EEG-child has 325 epochs.
EEG-adult has 316 epochs.


Well, there should be the same amount of epochs in each file. Moreover, when looking at the index of each epochs (see the x-axis of the plots bellow) we can see that they are all continuous, thus, not indicating which epochs were rejected:

In [5]:
# eeg1.plot()

In [6]:
# eeg2.plot()

### What's the plan now?

When loading an file in the EEGLAB format,  You have the following epoch indeces in your preprocessed file: 

`1, 2, 3, 4, 5, 6, 7`

And you know that the following epochs were rejected:

`3, 7, 8`

but then get 

`1, 2, 3, 4`

We'll now reconstruct the original epoch index as follows? (Within brackets):

`1(1), 2(2), NaN, 4(3), 5(4), 6(5), NaN, NaN, 9(6), 10(7)`


> **Careful, we have multiple round of rejection, so that method will have to be iterated over each round.**

In [7]:
df1 = eeg1.to_data_frame()
df2 = eeg2.to_data_frame()

In [8]:
Eeg1epochsIDs = df1.epoch.unique()
Eeg2epochsIDs = df2.epoch.unique()

## Make an example

### Example 1 (2 rounds of preprocessing)

In [9]:
df = pd.DataFrame({'Letters': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], 'indeces': [0, 1, 2, 3, 4, 5, 6, 7]})

# pd.set_option('display.max_rows', len(state_dict))
df

,Letters,indeces
0,A,0
1,B,1
2,C,2
3,D,3
4,E,4
5,F,5
6,G,6
7,H,7


In [10]:
# Now we remove two rows in a first round:
# Create a list of elements to remove
rmed_1 = [1, 3]

# Create a boolean mask indicating which rows to keep
mask = df['indeces'].isin(rmed_1)

# Remove the rows that match the elements in the list
df.drop(index=df[mask].index, inplace=True)

# Now we reset the index the same way saving this 'eeg' file would when being read for the next iteration's round 
df.indeces = [i for i in range(len(df))]
df

,Letters,indeces
0,A,0
2,C,1
4,E,2
5,F,3
6,G,4
7,H,5


In [11]:
# Now were remove three rows and directly reset the indeces
# Create a list of elements to remove
rmed_2 = [2, 5, 0]

# Create a boolean mask indicating which rows to keep
mask = df['indeces'].isin(rmed_2)

# Remove the rows that match the elements in the list
df.drop(index=df[mask].index, inplace=True)

df.indeces = [i for i in range(len(df))]

df

,Letters,indeces
2,C,0
5,F,1
6,G,2


Alrigth, now we have two list containning the indeces that were removed **`at the time of their round of rejection`**. 

Keep in mind that the index #4 could be deleted in multiple round as #4 could be reassigned when the file is re-read.

In [12]:
final_idx = df['indeces'].tolist()
print('Index that were rejected at the\n\t1st round: {}\n\t2st round: {}'.format(rmed_1, rmed_2))
print('The indeces as they are after the last rejection round {}'.format(final_idx))

Index that were rejected at the
	1st round: [1, 3]
	2st round: [2, 5, 0]
The indeces as they are after the last rejection round [0, 1, 2]


Define the list of epochs that were rejected as the `list` of `list` containing the IDs of the epochs that were rejected at each round of preprocessing. **The first `list` should contain the epochs IDs rejected at first preprocessing round and the last element should correspond to the last.** 

In [13]:
# Define the list of epochs that were rejected
rmed_list=[rmed_1, rmed_2]
rmed_list

[[1, 3], [2, 5, 0]]

In [14]:
updated_state_dict = align_utils.revert_to_original_idx(
    last_state = final_idx,
    removed_list  = rmed_list,
    verbose    = True)

Initial state:
	 0 0
	 1 1
	 2 2
Updated state:
	 0 NaN
	 1 0
	 2 NaN
	 3 1
	 4 2
	 5 NaN
Updated state:
	 0 NaN
	 2 0
	 4 NaN
	 5 1
	 6 2
	 7 NaN
	 1 NaN
	 3 NaN


Let's verify that we reconstructed the correspondance between IDs and their original place correctly.

In [15]:
letters=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

tt = sorted(updated_state_dict.keys())

print('\tOriginal ID\tFinal state\tLetter',)
for i in tt:
    print('\t',i,'\t\t',updated_state_dict[i],'\t\t', letters[i])  

	Original ID	Final state	Letter
	 0 		 NaN 		 A
	 1 		 NaN 		 B
	 2 		 0 		 C
	 3 		 NaN 		 D
	 4 		 NaN 		 E
	 5 		 1 		 F
	 6 		 2 		 G
	 7 		 NaN 		 H


Yay! We've rebuilt the connection between the indeces that were given post preprocessing (above, the column 'Original ID')and the ones we got (the 'Final state' column).

We now have a dictionnary that has `keys` representing each of the original epoch ID and `values` representing the state of that epoch at the end of preprocessing.

The **state** can be either
- `NaN` which indicated that this epoch was removed during preprocessing **or,**
- the id that was initially associated to the remaining epochs.

### Example 2 (3 rounds of preprocessing)

In [16]:
df2 = pd.DataFrame({'Letters': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], 'indeces': [0, 1, 2, 3, 4, 5, 6, 7]})
print('df2: original\n', df2)

rmed_1_2 = [1, 4]
df2.drop(index=df2[df2['indeces'].isin(rmed_1_2)].index, inplace=True)
df2.indeces = [i for i in range(len(df2))]
print('\ndf2: 1st round of rejection\n', df2)

rmed_2_2 = [0]
df2.drop(index=df2[df2['indeces'].isin(rmed_2_2)].index, inplace=True)
df2.indeces = [i for i in range(len(df2))]
print('\ndf2: 2nd round of rejection\n', df2)

rmed_3_2 = [3, 4]
df2.drop(index=df2[df2['indeces'].isin(rmed_3_2)].index, inplace=True)
df2.indeces = [i for i in range(len(df2))]
print('\ndf2: 3rd round of rejection\n', df2)


df2: original
   Letters  indeces
0       A        0
1       B        1
2       C        2
3       D        3
4       E        4
5       F        5
6       G        6
7       H        7

df2: 1st round of rejection
   Letters  indeces
0       A        0
2       C        1
3       D        2
5       F        3
6       G        4
7       H        5

df2: 2nd round of rejection
   Letters  indeces
2       C        0
3       D        1
5       F        2
6       G        3
7       H        4

df2: 3rd round of rejection
   Letters  indeces
2       C        0
3       D        1
5       F        2


In [17]:
updated_state_dict_2 = align_utils.revert_to_original_idx(
    last_state = df2['indeces'].tolist(),
    removed_list  = [rmed_1_2, rmed_2_2, rmed_3_2],
    verbose    = False)

In [18]:
print('\tOriginal ID\tFinal state\tLetter',)
for i in sorted(updated_state_dict_2.keys()):
    print('\t',i,'\t\t',updated_state_dict_2[i],'\t\t', letters[i])  

	Original ID	Final state	Letter
	 0 		 NaN 		 A
	 1 		 NaN 		 B
	 2 		 0 		 C
	 3 		 1 		 D
	 4 		 NaN 		 E
	 5 		 2 		 F
	 6 		 NaN 		 G
	 7 		 NaN 		 H


### Finding concurrent epochs from 

Alright, now we have 2 dictionnary containing as keys the original epochs index, and as values, the `last state`. I.e., either `NaN` if the epoch was removed **or** the latest index given to this epoch post-preprocessing. 

We're now going to see which paris of concurrent epochs are still available in each set of data.

In [20]:
print(updated_state_dict)
print(updated_state_dict_2)

{0: 'NaN', 2: 0, 4: 'NaN', 5: 1, 6: 2, 7: 'NaN', 1: 'NaN', 3: 'NaN'}
{2: 0, 3: 1, 5: 2, 6: 'NaN', 7: 'NaN', 0: 'NaN', 1: 'NaN', 4: 'NaN'}


First, we convert these dictionnary to pd.df:

In [34]:
df1 = pd.DataFrame.from_dict(
    updated_state_dict, 
    orient='index', 
    columns=['LastState-1']).sort_index()

df2 = pd.DataFrame.from_dict(
    updated_state_dict_2, 
    orient='index', 
    columns=['LastState-2']).sort_index()

print(df1, '\n\n', df2)

  LastState-1
0         NaN
1         NaN
2           0
3         NaN
4         NaN
5           1
6           2
7         NaN 

   LastState-2
0         NaN
1         NaN
2           0
3           1
4         NaN
5           2
6         NaN
7         NaN


then, we merge these `LastState` columns in the same `comparaisonDf`

In [41]:
comparaisonDf = df1.join(df2["LastState-2"])
comparaisonDf = comparaisonDf.replace('NaN', np.nan)
comparaisonDf

,LastState-1,LastState-2
0,NaN,NaN
1,NaN,NaN
2,0.0,0.0
3,NaN,1.0
4,NaN,NaN
5,1.0,2.0
6,2.0,NaN
7,NaN,NaN


Now drop the rows corresponding to the epochs that are not present in both eegs

In [44]:
comparaisonDf = comparaisonDf.dropna().astype(int)
comparaisonDf

,LastState-1,LastState-2
2,0,0
5,1,2


This df above now indicates which index have to be sampled to reconstruct the concurrence between the first EEG (`LastState-1`) and the second EEG (`LastState-2`)

## Now doing that with the `mne.Epoch` data

In [61]:
print([i for i in dyad])

[array(['213',
       'FINS_213_Child_FreePlay_Filt_xchan_epochs_rej_prunedICA_rej2.set',
       'FINS_213_ADULT_FreePlay_Rej_ICA_rej2.set'], dtype='<U64'), array(['205',
       'FINS_205_Child_FreePlay_xchan_epochs_rej_ica pruned_rej3.set',
       'FINS_205_Adult_FreePlay_xchan_rej_pruned with ICA_rej2.set'],
      dtype='<U64'), array(['217', 'FINS_217_Child_FreePlay_xchan_rej2.set',
       'FINS_217_Adult_FreePlay_xchan_rej2.set'], dtype='<U64'), array(['224',
       'FINS_224_Child_FreePlay_xchan_epochs_rej_ica _pruned_rej3.set',
       'FINS_224_Adult_FreePlay_xchan_epochs_ica_pruned_rej2.set'],
      dtype='<U64'), array(['220', 'FINS_220_Child_FreePlay_xchan_rej3.set',
       'FINS_220_Adult_FreePlay_xchan_ica_rej3.set'], dtype='<U64'), array(['206', 'FINS_206_Child_FreePlay_xchan_epochs_rej_ica_rej2.set',
       'FINS_206_Adult_FreePlay_xchan_epochs_ica_rej3.set'], dtype='<U64'), array(['222', 'FINS_222_Child_FreePlay_xchan_ICA_rej3.set',
       'FINS_222_Adult_FreePlay_xchan_i

## Trying with Dyad 213

### Extract preprocessed epoch ID for both file (i.e., last state)
First, let's take our dyad's files and extract their current epoch ID :

In [5]:
# For reference we'll be using 
print('Dyad: {}\nChild file: {}\nAdult file: {}'.format(dy[0], dy[1], dy[2]))

Dyad: 213
Child file: FINS_213_Child_FreePlay_Filt_xchan_epochs_rej_prunedICA_rej2.set
Adult file: FINS_213_ADULT_FreePlay_Rej_ICA_rej2.set


In [7]:
print('EEG-child has {} epochs.'.format(eeg_child.get_data().shape[0]))
print('EEG-adult has {} epochs.'.format(eeg_adult.get_data().shape[0]))

EEG-child has 325 epochs.
EEG-adult has 316 epochs.


In [6]:
child_late_state = eeg_child.events[:, 0]
adult_late_state = eeg_adult.events[:, 0]

In [24]:
# A way to subset the data

tt = eeg_adult.get_data()
print(tt.shape)
print(tt[[2, 25],:,:].shape)


(316, 18, 500)
(2, 18, 500)


In [26]:
eeg_adult.to_data_frame()

,time,condition,epoch,P7,P4,Cz,Pz,P3,P8,O1,...,T8,F8,C4,Fp2,Fz,F3,Fp1,T7,F7,Oz
0,0,1/1,0,-12.439453,1.142805,-10.955218,-9.090399,-11.991937,-30.485649,3.634571,...,0.020473,14.368764,-1.154086,22.301815,0.223805,5.535477,18.833206,3.650221,10.648489,-18.540926
1,2,1/1,0,-4.765141,5.739458,-3.855807,-3.456626,-6.451349,-23.083595,8.091072,...,2.895975,14.504980,3.621101,25.967552,6.214095,12.386865,26.934822,9.801796,18.559805,-12.466508
2,4,1/1,0,2.324517,9.967653,2.769588,1.917543,-1.405067,-15.273561,12.402609,...,5.515651,14.327770,8.039848,28.897728,11.563794,18.410034,33.790066,14.822166,24.956415,-6.494979
3,6,1/1,0,8.252581,13.506267,8.431911,6.629643,2.741567,-7.630542,16.221558,...,7.723204,13.943891,11.783163,30.919964,15.879862,23.123606,38.867527,18.301508,29.335447,-1.113773
4,8,1/1,0,12.604696,16.127539,12.772361,10.372193,5.718551,-0.704463,19.274471,...,9.423502,13.495095,14.622331,31.981352,18.906019,26.218401,41.866360,20.046278,31.476065,3.269217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157995,990,1,315,-1.366750,7.029265,-4.048895,-2.119711,-4.755799,-4.839591,-1.023677,...,3.815301,14.752668,4.161595,18.244930,4.128047,-0.319883,11.547513,-1.165699,-2.215918,-4.001302
157996,992,1,315,-0.486478,8.446573,-1.672904,-0.127561,-3.604706,-2.074218,0.833252,...,4.849911,16.060059,5.983804,21.306108,6.438555,2.049780,15.068205,-1.245648,-1.403956,-2.479939
157997,994,1,315,0.806278,10.377870,1.189085,2.308568,-2.192944,1.534577,2.955684,...,6.201814,17.504497,8.254701,24.437096,9.092056,4.572440,18.706781,-1.091839,-0.516623,-0.494205
157998,996,1,315,2.352508,12.650732,4.281524,4.976461,-0.643294,5.621661,5.159948,...,7.724286,18.930176,10.771629,27.353302,11.841879,7.024847,22.128534,-0.782283,0.286296,1.788580


Get the `last state ID` of the adult and child's EEG epochs:

In [29]:
epo_id_adult = [epoch_idx for epoch_idx, _ in enumerate(eeg_adult)]
epo_id_child = [epoch_idx for epoch_idx, _ in enumerate(eeg_child)]

Define the list of list showing which epochs were removed at each round:

In [31]:
def make_list_from_text(unformated_list):
    """
    Return a flattened list of integers from a str formated as:
    '1 3 24:27 234' --> [1, 3, 24, 25, 26, 27, 234].
    """
    tmp = []
    for item in [i for i in unformated_list.split(' ')] :
        if ':' in item:
            start, end = item.split(':')
            tmp += list(range(int(start), int(end)+1))
        else:
            tmp.append(int(item))
    return tmp

[17, 25, 26, 31, 36, 37, 41, 42, 43, 50, 51, 83, 84, 116, 117, 152, 153, 156, 157, 158, 163, 164, 165, 171, 231, 232, 244, 245, 247, 248, 253, 254, 255, 261, 268, 269, 270, 271, 272, 306, 307, 310, 311, 314, 316, 318, 319, 320, 335, 338, 339, 371, 379, 380]


In [62]:
round1_child = make_list_from_text('17 25 26 31 36 37 41 42 43 50 51 83 84 116 117 152 153 156 157 158 163 164 165 171 231 232 244 245 247 248 253 254 255 261 268 269:272 306 307 310 311 314 316 318 319 320 335 338 339 371 379 380')
round2_child = make_list_from_text('135 137 147 148:150 159 194 211 223 226 227 288 310 314 319 325 341 342')
child_rej = [round1_child, round2_child]
total_rej_child = len(round1_child) + len(round2_child)

# round1_adult = make_list_from_text('9 10 58 60 61:63 76 78 79 80 83 84 85 90 103 104 105 112 114 115 132 133:135 153 155 156 173 174 182 226 227 248 249 250 253 257 259:263 342 348 356 357 363 364:367 370 383 391 393')
round1_adult = make_list_from_text('9 10 58 60 61:63 76 78 79 80 83 84 85 90 103 104 105 112 114 115 132 133:135 153 155 156 173 174 182 226 227 248 249 250 253 257 259 263 342 348 356 357 363 364:367 370 383 391 393')
round2_adult = make_list_from_text('2 9 10 11 33 46 50 60 68 76 101 103 128 139 142 145 187 192 222 223 319 321 322 323 325 331 335 343')
adult_rej = [round1_adult, round2_adult]
c = len(round1_adult) + len(round2_adult)

In [52]:
assert eeg_child.get_data().shape[0] + total_rej_child == eeg_adult.get_data().shape[0] + total_rej_adult, 'there is an issue with the total amount of epochs\nChild would have {} total while Adult would have {}'.format(
    total_rej_child +eeg_child.get_data().shape[0], total_rej_adult+eeg_adult.get_data().shape[0]
)

AssertionError: there is an issue with the total amount of epochs
Child would have 398 total while Adult would have 397

### Let's try with another one (dyad 205)

In [54]:
dy = dyad[1]
print('Dyad: {}\nChild file: {}\nAdult file: {}'.format(dy[0], dy[1], dy[2]))

Dyad: 205
Child file: FINS_205_Child_FreePlay_xchan_epochs_rej_ica pruned_rej3.set
Adult file: FINS_205_Adult_FreePlay_xchan_rej_pruned with ICA_rej2.set


In [55]:
eeg_child = align_utils.EpochsEEGLAB_to_mneEpochsFIF('{}{}_{}_FP/{}'.format(data_path, dy[0], 'child', dy[1])) 
eeg_adult = align_utils.EpochsEEGLAB_to_mneEpochsFIF('{}{}_{}_FP/{}'.format(data_path, dy[0], 'adult', dy[2]))

Extracting parameters from /Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/../FINS-data/205_child_FP/FINS_205_Child_FreePlay_xchan_epochs_rej_ica pruned_rej3.set...
Not setting metadata
251 matching events found
No baseline correction applied
0 projection items activated
Ready.
Reading /var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/tmp9ukp09lltmp.fif ...


/Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/utils/align_utils.py:29: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  tmp = mne.io.read_epochs_eeglab(path)
/Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/utils/align_utils.py:33: RuntimeWarning: This filename (/var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/tmp9ukp09lltmp.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  tmp.save(tmpdir+"tmp.fif", overwrite=True, verbose=None)
/Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/utils/align_utils.py:36: RuntimeWarning: This filename (/var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/tmp9ukp09lltmp.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  return mne.read_epochs(tmpdir+"tmp.fif")


    Found the data of interest:
        t =       0.00 ...     998.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
251 matching events found
No baseline correction applied
0 projection items activated
Extracting parameters from /Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/../FINS-data/205_adult_FP/FINS_205_Adult_FreePlay_xchan_rej_pruned with ICA_rej2.set...
Not setting metadata
147 matching events found
No baseline correction applied
0 projection items activated
Ready.
Reading /var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/tmporwyrm6ltmp.fif ...
    Found the data of interest:
        t =       0.00 ...     998.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
147 matching events found
No baseline correction applied
0 projection items activated


/Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/utils/align_utils.py:29: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  tmp = mne.io.read_epochs_eeglab(path)
/Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/utils/align_utils.py:33: RuntimeWarning: This filename (/var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/tmporwyrm6ltmp.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  tmp.save(tmpdir+"tmp.fif", overwrite=True, verbose=None)
/Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/utils/align_utils.py:36: RuntimeWarning: This filename (/var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/tmporwyrm6ltmp.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  return mne.read_epochs(tmpdir+"tmp.fif")


In [56]:
print('EEG-child has {} epochs.'.format(eeg_child.get_data().shape[0]))
print('EEG-adult has {} epochs.'.format(eeg_adult.get_data().shape[0]))

EEG-child has 251 epochs.
EEG-adult has 147 epochs.


Now creating the updated dictionnary

In [50]:
child_state_dict = align_utils.revert_to_original_idx(
    last_state = epo_id_child,
    removed_list  = child_rej,
    verbose    = False)
# print(child_state_dict)
print(len(child_state_dict.keys()))

adult_state_dict = align_utils.revert_to_original_idx(
    last_state = epo_id_adult,
    removed_list  = adult_rej,
    verbose    = False)
# print(adult_state_dict)
print(len(adult_state_dict.keys()))

398
403


# Notes

Dyad 213 does not have the same amout of epoch i.e., sum(epochs, rej1, rej2, rej3)
Dyad 205 is intialy not equilibrated 

# Useful references

- To get comfortable with the MNE documentation, you should know that MNE is based on python [Object Oriented Programming (00P)](hstate_dictps://realpython.com/python3-object-oriented-programming/). These objects are defined from a python `Class`.
    - You can get familiarized with the OOP structure and its componenent, e.g. `methods` (a function associated to the the object) and `astate_dictribute` (a variable associated to the object), wit [this tutorial](hstate_dictps://www.datacamp.com/tutorial/python-oop-tutorial).
    - In MNE, we find [`Raw` objects](hstate_dictps://mne.tools/stable/generated/mne.io.Raw.html) (continuous data) or [`Epoch` objects](hstate_dictps://mne.tools/stable/generated/mne.Epochs.html) (a collection of epochs). 

You can find an introduction to the **Epochs data structure** [here](hstate_dictps://mne.tools/stable/auto_tutorials/epochs/10_epochs_overview.html) in MNE. 

### Extracting the epoch data

We're now going to extract the epoch data from the mne.EpochFIF to apply the operation described above.

In [67]:
dyad

[array(['213',
        'FINS_213_Child_FreePlay_Filt_xchan_epochs_rej_prunedICA_rej2.set',
        'FINS_213_ADULT_FreePlay_Rej_ICA_rej2.set'], dtype='<U64'),
 array(['205',
        'FINS_205_Child_FreePlay_xchan_epochs_rej_ica pruned_rej3.set',
        'FINS_205_Adult_FreePlay_xchan_rej_pruned with ICA_rej2.set'],
       dtype='<U64'),
 array(['217', 'FINS_217_Child_FreePlay_xchan_rej2.set',
        'FINS_217_Adult_FreePlay_xchan_rej2.set'], dtype='<U64'),
 array(['224',
        'FINS_224_Child_FreePlay_xchan_epochs_rej_ica _pruned_rej3.set',
        'FINS_224_Adult_FreePlay_xchan_epochs_ica_pruned_rej2.set'],
       dtype='<U64'),
 array(['220', 'FINS_220_Child_FreePlay_xchan_rej3.set',
        'FINS_220_Adult_FreePlay_xchan_ica_rej3.set'], dtype='<U64'),
 array(['206', 'FINS_206_Child_FreePlay_xchan_epochs_rej_ica_rej2.set',
        'FINS_206_Adult_FreePlay_xchan_epochs_ica_rej3.set'], dtype='<U64'),
 array(['222', 'FINS_222_Child_FreePlay_xchan_ICA_rej3.set',
        'FINS_222_Adul